In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from scipy import io
from scipy.spatial.transform import Rotation as R
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import sys
sys.path.append("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1")

from rotplot import rotplot

In [4]:
# Loading Accelerometer bias and scale Data
params = io.loadmat("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/IMUParams.mat")
s = params['IMUParams'][0]
ba = params['IMUParams'][1]

In [5]:
# Loading Vicon Data
vicon1_data = io.loadmat("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot1.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot2.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot3.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot4.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot5.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/Vicon/viconRot6.mat")
rots1 = vicon1_data['rots'] # 3 x 3 x 5561 Data
ts_vicon1 = vicon1_data['ts'] # 1 x 5561 Data
rots_reshaped1 = rots1.transpose(2,0,1).reshape(rots1.shape[2],3,3) # 5561 x 3 x 3 Data
euler_angles_vicon1 = np.zeros((3,rots1.shape[2])) # 3 x 5561 Data

for i in range(rots1.shape[2]):
  R = rots_reshaped1[i]
  z1 = np.arctan2(R[1,0],R[0,0]) # Yaw
  y1 = np.arctan2(-R[2,0], np.sqrt(R[0,0]**2 + R[1,0]**2))#np.arcsin(R[2,0])         # Pitch
  x1 = np.arctan2(R[2,1],R[2,2]) # Roll
  euler_angles_vicon1[:,i] = [z1,y1,x1]

z_vicon1 = euler_angles_vicon1[0,:][:,np.newaxis]
y_vicon1 = euler_angles_vicon1[1,:][:,np.newaxis]
x_vicon1 = euler_angles_vicon1[2,:][:,np.newaxis]


In [6]:
# Reading the Data
imu1_data = io.loadmat("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw1.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw2.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw3.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw4.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw5.mat")
# ("/content/drive/MyDrive/WPI/NJS-Autonomous-Aerial-Vehicles/sbachimanchi_p0/Phase1/Data/Train/IMU/imuRaw6.mat")
vals1 = imu1_data['vals'] # Raw Data
ts_imu1 = imu1_data['ts']
ax1 = vals1[0,:]
ay1 = vals1[1,:]
az1 = vals1[2,:]
wz1 = vals1[3,:]
wx1 = vals1[4,:]
wy1 = vals1[5,:]
# ax_new1 = (ax1 + ba[0])/s[0]
# ay_new1 = (ay1 + ba[1])/s[1]
# az_new1 = (az1 + ba[2])/s[2]
ax_new1 = (ax1*s[0] + ba[0])*9.8
ay_new1 = (ay1*s[1] + ba[1])*9.8
az_new1 = (az1*s[2] + ba[2])*9.8
samples1 = 500
bgz1 = np.mean(wz1[:samples1])
bgx1 = np.mean(wx1[:samples1])
bgy1 = np.mean(wy1[:samples1])
wz_new1 = (3300/1023)*(np.pi/180)*0.3*(wz1-bgz1)
wx_new1 = (3300/1023)*(np.pi/180)*0.3*(wx1-bgx1)
wy_new1 = (3300/1023)*(np.pi/180)*0.3*(wy1-bgy1)
vals_new1 = np.row_stack((ax_new1, ay_new1, az_new1, wz_new1, wx_new1, wy_new1)) # 6x5645 Data in SI Units
w_new1 = np.row_stack((wz_new1, wx_new1, wy_new1)) # 3 x 5645 Angular Velocity Data in SI Units
a_new1 = np.row_stack((ax_new1, ay_new1, az_new1)) # 3 x 5645 Angular Velocity Data in SI Units

In [7]:
# Function for orientation based on Gyro Data
def orientation_gyro(euler_angles_initial, angular_velocity, time):
  orientation = np.zeros((3,angular_velocity.shape[1]))
  orientation[:,0][:,np.newaxis] = euler_angles_initial
  for i in range(1, angular_velocity.shape[1]):
    orientation[:,i] = orientation[:,i-1] + angular_velocity[:,i]*(time[:,i]-time[:,i-1])
  return orientation


In [8]:
# Function for orientation based on Accelerometer Data
def orientation_accelero(acc_data):
  ax = (a_new1[0,:]) # Roll
  ay = (a_new1[1,:]) # Pitch
  az = (a_new1[2,:]) # Yaw
  psi = np.arctan2(np.sqrt(ax**2 + ay**2),az) # Yaw
  theta = np.arctan2(-ax,np.sqrt(ay**2 + az**2)) # Pitch
  phi = np.arctan2(ay,np.sqrt(ax**2 + az**2)) # Roll
  orientation = np.row_stack((psi, theta, phi))
  return orientation

In [9]:
# Function for simple complementary filter
def complementary_filter(gyro_orientation, accelero_orientation, time, alpha):
  filtered_orientation = np.zeros((3,gyro_orientation.shape[1]))

  # High pass filter for gyro orientation
  high_pass_filtered_data = np.zeros((3,gyro_orientation.shape[1]))
  high_pass_filtered_data[:,0] = gyro_orientation[:,0]
  for i in range(1, gyro_orientation.shape[1]):
    high_pass_filtered_data[:,i] = ((1-alpha)*high_pass_filtered_data[:,i-1]) + ((1-alpha)*(gyro_orientation[:,i]-gyro_orientation[:,i-1]))

  # Low pass filter for accelero orientation
  low_pass_filtered_data = np.zeros((3,accelero_orientation.shape[1]))
  low_pass_filtered_data[:,0] = accelero_orientation[:,0]
  for i in range(1, accelero_orientation.shape[1]):
    low_pass_filtered_data[:,i] = ((1-alpha)*accelero_orientation[:,i]) + (alpha*low_pass_filtered_data[:,i-1])

  filtered_orientation = (1-0.7)*high_pass_filtered_data + 0.7*low_pass_filtered_data

  return filtered_orientation

In [10]:
# Function for converting ZYX euler angles into Quaternion
def ZYXeuler_quaternion(yaw, pitch,roll):
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)

    qw = cy * cp * cr + sy * sp * sr
    qx = cy * cp * sr - sy * sp * cr
    qy = sy * cp * sr + cy * sp * cr
    qz = sy * cp * cr - cy * sp * sr

    return np.array([qw, qx, qy, qz])

In [46]:
# Function for Quarternion Multiplication
def quaternion_multiplication(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2

    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2

    return np.array([w, x, y, z])

In [55]:
# Function for Normalized Vector calculation
def normalized_vector(x):
  norm_x = x/np.linalg.norm(x)
  return norm_x

In [47]:
# Orientation increment from Gyro Data
# euler_angles_initial = euler_angles_vicon1[:,0][:,np.newaxis]
# initial_z = euler_angles_initial[0]
# initial_y = euler_angles_initial[1]
# initial_x = euler_angles_initial[2]

# orientation_gyroscope_quat = ZYXeuler_quaternion(initial_z,initial_y,initial_x)

# orientation_gyroscope = orientation_gyro(euler_angles_initial,w_new1,ts_imu1)
# z_gyro = orientation_gyroscope[0,:] # Yaw
# y_gyro = orientation_gyroscope[2,:] # Pitch
# x_gyro = orientation_gyroscope[1,:] # Roll

# w_measured = np.row_stack((np.zeros_like(x_gyro),x_gyro,y_gyro,z_gyro))

# quat_rate = np.zeros((4,orientation_gyroscope.shape[1]))
# normalized_quat = np.zeros((4,orientation_gyroscope.shape[1]))
# quat_rate[:,0] = orientation_gyroscope_quat
# normalized_quat[:,0] = quat_rate[:,0]/np.linalg.norm(orientation_gyroscope_quat)






# for i in range(1, orientation_gyroscope.shape[1]):
#   normalized_quat = (0.5)*(quaternion_multiplication(normalized_vector(q_est[:,i]),




# normalized_quat = np.zeros((4,orientation_gyroscope.shape[1]))
# for i in range(orientation_gyroscope_quat.shape[1]):
#   normalized_quat[:,i] = orientation_gyroscope_quat[:,i]/np.linalg.norm(orientation_gyroscope_quat[:,i])

# quat_rate = np.zeros((4,orientation_gyroscope.shape[1]))
# for i in range(1, orientation_gyroscope_quat.shape[1]):
#     quat_rate[:,i] = (0.5)*(quaternion_multiplication(normalized_quat[:,i-1], w_measured[:,i]))



In [76]:
euler_angles_initial = euler_angles_vicon1[:,0][:,np.newaxis]
orientation_gyroscope = orientation_gyro(euler_angles_initial,w_new1,ts_imu1)
orientation_gyroscope = np.row_stack((np.zeros_like(orientation_gyroscope[0,:]),orientation_gyroscope))

q_hat_est = np.zeros((4,orientation_gyroscope.shape[1]))
qw_dot = np.zeros((4,orientation_gyroscope.shape[1]))
J_transpose = np.zeros((4,orientation_gyroscope.shape[1]))
f = np.zeros((4,orientation_gyroscope.shape[1]))
grad_desc = np.zeros((4,orientation_gyroscope.shape[1]))
qest_dot = np.zeros((4,orientation_gyroscope.shape[1]))
q_next_est = np.zeros((4,orientation_gyroscope.shape[1]))

q_hat_est[:,0] = np.array([1, 0, 0, 0])

for i in range(1, orientation_gyroscope.shape[1]):
  qw_dot[:,i] = (0.5)*quaternion_multiplication(normalized_vector(q_hat_est[:,i-1]),orientation_gyroscope[:,i])
  J_transpose[:,i] = np.array([[-2*q_hat_est[2,i], 2*q_hat_est[1,i], 0],
                   [2*q_hat_est[3,i], 2*q_hat_est[0,i], -4*q_hat_est[1,i]],
                   [-2*q_hat_est[0,i], 2*q_hat_est[3,i], -4*q_hat_est[2,i]],
                   [2*q_hat_est[1,i], 2*q_hat_est[2,i], 0]])
  f[:,i] = np.array([2*((q_hat_est[1,i]*q_hat_est[3,i])-(q_hat_est[0,i]*q_hat_est[2,i]))-ax_new1[:,i]],
                    [2*((q_hat_est[0,i]*q_hat_est[1,i])+(q_hat_est[2,i]*q_hat_est[3,i]))-ay_new1[:,i]],
                    [2*(0.5-(q_hat_est[1,i]**2)-(q_hat_est[2,i]**2))-az_new1[:,i]])
  grad_desc[:,i] = -0.6*normalized_vector(f[:,i])
  qest_dot[:,i] = qw_dot[:,i] + grad_desc[:,i]
  q_next_est[:,i] = qest_dot[:,i] + qest_dot[:,i]*(ts_imu1[:,i]-ts_imu1[:,i-1])
  q_hat_est[:,i] = q_next_est[:,i]






ValueError: ignored

In [62]:
q_hat_est = np.zeros((4,orientation_gyroscope.shape[1]))
q_hat_est[:,0] = np.array([1, 0, 0, 0])

In [65]:
q_hat_est[0,0]

1.0

In [66]:
nax_new1)

(5645,)